# Project 2
## Eniacs product strategy

Your analyse can finally settle an ongoing debate for the mgmt: whether or not it’s beneficial to discount products.

*Marketing opinion*\
Marketing Team Lead is convinced that offering discounts is beneficial in the long run:
1. discounts improve customer acquisition, satisfaction and retention, and allows the company to grow.

*Investors opinion*\
Main investors in the Board are worried about offering aggressive discounts:
1. recent quarterly results showed an increase in number of orders, but a decrease in the total revenue
2. focus in quality segment, rather than offer lowest prices in market

*Your analytical and business skills are needed to give clarity in the following aspects:*
- How products should be classified into different categories in order to simplify reports and analysis.
- What is the distribution of product prices across different categories.
- How many products are being discounted.
- How big are the offered discounts as a percentage of the product prices.
- How seasonality and special dates (Christmas, Black Friday) affect sales.
- How could data collection be improved.

`Present to company board your analysis (short and concise, but also engaging and convincing)`

In [271]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
pd.options.display.float_format = '{:.2f}'.format

In [272]:
path = 'eniac/clean/'
orderlines = pd.read_csv(path + 'orderlines_cl.csv')
orders = pd.read_csv(path + 'orders_cl.csv')
brands = pd.read_csv(path + 'brands_cl.csv')
products = pd.read_csv(path + 'products_cl.csv')

In [273]:
orders['created_date'] = pd.to_datetime(orders['created_date'])
orderlines['date'] = pd.to_datetime(orderlines['date'])

## Part from Rias

In [ ]:
p_df2 = (
df2
    .assign(year = df2['date'].dt.strftime('%Y'),
            month = df2['date'].dt.strftime('%m'))
    .groupby(['year', 'month'])
    .agg({'total_paid':'sum'})
    .reset_index()
    .assign(qlty_perc = lambda x: np.where(x['unit_price_cat'].str.count('quality')==1,1 - x['total_price'] / x['total_paid']*100, '0'))
)
p_df2

In [301]:
p_df2 = (
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed"')
    .assign(price_cat = lambda x: pd.cut(x['unit_price'], [0, 1000, 8500]),
            y_m = lambda x: x['date'].dt.to_period('M'))
    .groupby(['y_m','price_cat'])
    .agg(count = ('id_order','count'))
    .reset_index()
    .assign(total_count = lambda x: x.groupby('y_m')['count'].transform('sum'))
    .assign(ratio = lambda x: x['count'] / x['total_count'],
            price_cat = lambda x: x['price_cat'].astype('str'))    
    .query('price_cat == "(1000, 8500]"')
#     .to_excel('eniac/filename.xlsx')
)
p_df2

In [ ]:
plt.subplots(figsize=(10, 6))
plt.plot_date(data=p_df2, x='y_m', y='ratio', linestyle='-')
plt.title('Ratio of quality products (+500 euro) on total monthly sales 17/18')
plt.ylabel('Ratio in %')
plt.ylim(0, p_df['ratio'].max()*1.2)
sns.despine()
plt.show()

In [ ]:
orderlines['id_order'].value_counts().head(10)

In [340]:
t = (
orderlines
    .query('sku == "APP1922"')
    .merge(products, how='left')
    ['name']
)   
t.head(1)
    #['unit_price'].describe()
## (sku – stock keeping unit: a unique identifier for each product)
## amount of APP1922 sku: 1610, min: 0, max: 179, mean: 162.41, std: 11.56

0    AirPods Apple Bluetooth Headset for iPhone iPa...
Name: name, dtype: object

In [ ]:
(
orderlines
    .query('id_order == 395611')
    .merge(products, how='left')
#     ['name']
)